# 1. Overfitting
 - 데이터를 2차원의 좌표 상에 나타냈을 때, 학습의 목표는 데이터에 맞는 선/곡선을 잘 fitting 시키는 것이다.
 - Underfitting : 학습 데이터를 잘 학습하지 못함
 - Overfitting : 학습 데이터를 과하게 학습함

# 2. Overfitting의 문제점
 - Overfitting이 일어나면 학습 데이터는 완벽하게 분류해 내지만, 학습 데이터의 케이스에만 너무 과적합 되어 있어서 새로운 데이터가 들어오면 잘 분류하지 못한다.
 - Overfitting이 일어났을 때 Train 데이터 셋에서는 정확도가 굉장히 높고, Test 데이터 셋서는 정확도가 높지 못하다.

# 3. Overfitting 해결 방법
 - Training data를 더 구한다.
 - feature의 수를 줄인다.
 - Regularization
 - Dropout : 레이어 안의 각 노드들 중 일정 노드를 설정 값에 따라 랜덤하게 삭제한다.

# 4. Dropout이란?
 - '학습'을 할 때 레이어 안의 각 노드들 중 일정 노드를 설정 값에 따라 랜덤하게 삭제(드랍)한다.
 - 성능의 향상을 가져오고, 오버피팅을 방지한다.
 - Backpropagation의 매 업데이트마다 다시 랜덤으로 노드를 선택하므로, 매 업데이트마다 새로운 네트워크를 이용하는 것과 같다. (여러 네트워크들이 결과를 내는, 네트워크 앙상블의 효과!)
 - ex) 한 레이어가 784개의 노드로 구성되어 있는데, dropout 비율을 0.5로 하면 그 레이어의 노드를 대략 392개만 이용한다.

# 5. Dropout 과정
### (1) 기존의 NN
 - (1) 인풋 X가 들어온다.
 - (2) Hidden layer의 각 노드들이 Weight와 bias를 계산한 뒤 활성화 함수를 통해 다음 레이어로 결과값을 출력한다.
 - (3) 아웃풋 Y를 구한다.
 - (4) GT와 Y를 비교해 Loss를 구하고, BP를 한다.
 - (5) 최종적으로 Test set을 통해 학습된 모델의 정확도를 평가한다.
 
 <br>
 
### (2) Overfitting 적용한 NN
 - (1) 인풋 X가 들어온다.
 - (2) Hidden layer의 각 노드들 중 일정 노드는 설정 값에 따라 랜덤하게 삭제된다.
 - (3) Hidden layer의 각 노드들 중 삭제되지 않은 노드의 Weight와 bias만 계산한 뒤 활성화 함수를 통해 다음 레이어로 결과값을 출력한다.
 - (4) 아웃풋 Y를 구한다.
 - (5) GT와 Y를 비교해 Loss를 구하고, BP를 한다. (BP를 할 때도 Dropout이 적용되어 매번 업데이트마다 랜덤으로 사용할 노드들을 다시 선택한다. 그 노드들의 Weight와 bias만을 계산함)
 - (6) 최종적으로 Test set을 통해 학습된 모델의 정확도를 평가하는데, 이 때는 Dropout을 적용하지 않고 원래의 온전한 노드들로 평가한다!

# 6. MNIST NN Dropout 실습

In [1]:
import torchvision.datasets as dsets # torchvision의 datasets를 불러와서 dsets로 이용
import torch
import torchvision.transforms as transforms

device = 'cuda' if torch.cuda.is_available() else 'cpu' # cuda가 사용 가능한 상황이면 cuda를 쓰고, 아니면 CPU를 사용

# 데이터 불러오기
mnist_train = dsets.MNIST(root = "MNIST_data/", train = True, transform = transforms.ToTensor(), download = True) # MNIST 함수를 이용함. root로 경로 지정, train으로 train set 불러올지 설정, transform으로 MNIST 일반 이미지를 PyTorch가 이미지 인식하는 방법대로 읽어오게 설정. download로 root에 MNIST 데이터가 없으면 다운을 받아오게 설정.
mnist_test = dsets.MNIST(root = "MNIST_data/", train = False, transform = transforms.ToTensor(), download=True)

# 불러온 데이터 사용하기
batch_size = 128
data_loader = torch.utils.data.DataLoader(dataset = mnist_train, batch_size = batch_size, shuffle=True, drop_last = True) # 데이터를 실제로 사용하기 위해서 DataLoader를 이용한다. DataLoader로 어떤 데이터를 로드 할 지 설정, batch_size로 총 60,000개의 데이터를 몇개 씩 잘라서 가져올 지 설정, shuffle로 데이터를 랜덤 순서로 불러올지 설정, drop_last로 batch_size로 짤라서 불러 올 때 맨 마지막에 남은 찌끄러기들을 사용 안 할지 설정

linear1 = torch.nn.Linear(784, 512, bias = True)
linear2 = torch.nn.Linear(512, 512, bias = True)
linear3 = torch.nn.Linear(512, 512, bias = True)
linear4 = torch.nn.Linear(512, 512, bias = True)
linear5 = torch.nn.Linear(512, 10, bias = True)
relu = torch.nn.ReLU()
drop_prob = 0.3
dropout = torch.nn.Dropout(p=drop_prob) # 드랍아웃! p를 통해 삭제할 비율을 설정한다.

# 모델에 dropout을 넣어 주면, 해당 레이어에 dropout을 적용할 수 있다.
model = torch.nn.Sequential(linear1, relu, dropout, # linear1 레이어에 dropout 적용!
                            linear2, relu, dropout, # linear2 레이어에 dropout 적용!
                            linear3, relu, dropout, # linear3 레이어에 dropout 적용!
                            linear4, relu, dropout, # linear4 레이어에 dropout 적용!
                            linear5).to(device)

# 주의! Dropout은 '학습'할 때만 적용하고, 'Test'할 때는 Dropout을 적용 안 한 원래의 온전한 노드들로 평가하기 때문에, 모델을 train 모드와 eval 모드로 나눠야 한다.
training_epochs = 15                        
total_batch = len(data_loader)
model.train() # 모델의 Train 모드! Dropout 적용!(Dropout = True)
for epoch in range(training_epochs):
    pass

with torch.no_grad():
    model.eval() # 모델의 평가 모드! Dropout 미적용!(Dropout = False)
